In [1]:
test = {}
print(test)

In [ ]:
test["test"] = []
print(test)

In [6]:
import json

# open the results file
with open("results/results.json", "r") as f:
    results = json.load(f)

scrfd_results = {}
for model_name, results_list in results.items():
    # first do the conversion
    results_list = [
        {
            "image": x["image"],
            "time": x["time"],
            "error rate": 100
            - (
                (x["faces found"] * 100)
                / (x["faces found"] + x["faces not found"])
            ),
            "height": x["height"],
            "width": x["width"],
        }
        for x in results_list
    ]
    # now add the results to the scrfd_results dictionary
    scrfd_results[model_name] = results_list

results = scrfd_results["10g_kps"]
print(type(results))



<class 'list'>


In [4]:
import cv2
import os
import glob


# for all jpg in the dataset folder, make a set of all existing sizes of image
# this is to make sure that we don't resize an image to a size that already exists

# get all jpg files in the dataset folder
images = glob.glob(os.path.join("dataset", "*.jpg"))

print(len(images))
# get the sizes of all images
sizes = set()
for image in images:
    img = cv2.imread(image)
    sizes.add((img.shape[0], img.shape[1]))
print(sizes)

219
{(1718, 2444)}


In [5]:
import cv2
import numpy as np
import glob
import os
import json
from time import time

from mmdet.apis import inference_detector, init_detector


def detect_faces_scrfd(image_path, model, score_thr=0.3):
    # read the image
    img = cv2.imread(image_path)
    # perform inference
    start = time()
    result = inference_detector(model, img)
    duration = time() - start
    # remove all detections with a score lower than the threshold
    result = result[0][result[0][:, 4] > score_thr]
    # count the number of faces
    faces = result.shape[0]
    # draw the bounding boxes
    if len(result) > 0:
        for array in result:
            # get the coordinates of the bounding box
            x1, y1, x2, y2 = array[0], array[1], array[2], array[3]
            # draw the bounding box
            cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 5)
    # return the image with the detections
    return img, duration, faces


images_path = "./data/retinaface/test/images"
result_path = "./results"
models_path = "./models"

model = {"config": "scrfd_10g_custom.py", "model": "custom/latest.pth"}

# get all images in the data/retinaface/test/images folder. All images are jpg in subfolders
images = glob.glob(os.path.join(images_path, "**/*.jpg"), recursive=True)
print(len(images))

22


In [6]:
checkpoint_path = os.path.join(models_path, model["model"])
config_path = os.path.join("./configs/scrfd", model["config"])
model = init_detector(config_path, checkpoint_path, device="cuda:0")

Use load_from_local loader


In [7]:
import matplotlib.pyplot as plt
import numpy as np

In [8]:
# for image in images:
#     # perform face detection on the image, using the SCRFD_2.5G_KPS model
#     result, duration, faces = detect_faces_scrfd(image, model, score_thr=0.4)
#     # print the image (make it so that it is not blue)
#     plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
#     plt.show()

# for the first 6 images in the dataset, perform face detection on the image, using the model, and save them with name inisghtface_dataset_results_1.png, insightface_dataset_results_2.png, etc.
for i in range(6):
    # perform face detection on the image, using the SCRFD_2.5G_KPS model
    result, duration, faces =detect_faces_scrfd(images[i], model, score_thr=0.4)
    # save the image
    plt.imsave(
        f"insightface_dataset_results_{i+1}.png",
        cv2.cvtColor(result, cv2.COLOR_BGR2RGB),
    )
